In [32]:
%%capture 
!pip install h2o

In [33]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os

In [34]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML

In [35]:
# If this doesnt work - install java. 
# Start the H2O cluster (locally)
h2o.init(ip="localhost", min_mem_size_GB=8)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,59 mins 39 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_ueb0ml
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.979 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [36]:
# Load ML dataframe
df_ML  = pd.read_csv(os.getcwd() + '/sample_data/signal_peptides_ML.csv')
df_ML

,Peptide sequences,end_pos,MM mean abundances (Scaled-Normalized),MM + Nitrogen mean abundances (Scaled-Normalized),MM mean abundances (Raw-Normalized),MM + Nitrogen mean abundances (Raw-Normalized)
0,MMVAWWSLFLYGLQVAAPAL,20,0.058537,0.058537,1.000000,1.000000
1,MEAFNLHNFLSSLYILLPFVILANPVH,27,0.059285,0.059285,0.010224,0.417923
2,MLRVSAIFMACLLLATAA,18,0.430035,0.430035,0.000144,0.339312
3,MAVRIARFLGLSTVAYLALANGID,24,0.414383,0.414383,0.000144,0.276919
4,MVSFSSCLRALALGSSVLAVQPVL,24,0.228295,0.228295,0.025872,0.218331
...,...,...,...,...,...,...
1056,MQVKLFYTLALWAPILVS,18,0.000000,0.000000,0.000000,0.000000
1057,MKSLIWALPFIPLAY,15,0.000000,0.000000,0.000000,0.000000
1058,MWPTRSLSSLFFLSLALGSPVS,22,0.000000,0.000000,0.000000,0.000000
1059,MLLPRLSSLLCLAGLATMPVAN,22,0.000000,0.000000,0.000000,0.000000


In [37]:
df_ML['default_rank'] = df_ML['MM mean abundances (Raw-Normalized)'].rank()
df_ML['max_rank'] = df_ML['MM mean abundances (Raw-Normalized)'].rank(method='max')
df_ML['NA_bottom'] = df_ML['MM mean abundances (Raw-Normalized)'].rank(na_option='bottom')
df_ML['pct_rank'] = df_ML['MM mean abundances (Raw-Normalized)'].rank(pct=True)
df_ML = df_ML.sort_values(by= 'max_rank', ascending = False)

In [38]:
df_ML

,Peptide sequences,end_pos,MM mean abundances (Scaled-Normalized),MM + Nitrogen mean abundances (Scaled-Normalized),MM mean abundances (Raw-Normalized),MM + Nitrogen mean abundances (Raw-Normalized),default_rank,max_rank,NA_bottom,pct_rank
0,MMVAWWSLFLYGLQVAAPAL,20,0.058537,0.058537,1.000000,1.000000,1061.0,1061.0,1061.0,1.000000
4,MVSFSSCLRALALGSSVLAVQPVL,24,0.228295,0.228295,0.025872,0.218331,1060.0,1060.0,1060.0,0.999057
7,MMVAWWSLFLYGLQVAAPAL,20,0.044270,0.044270,0.025062,0.017628,1059.0,1059.0,1059.0,0.998115
10,MYFNYAAATLAALLPLCS,18,0.020029,0.020029,0.019565,0.019066,1058.0,1058.0,1058.0,0.997172
11,MFIKYALPALAAAQAVF,17,0.031471,0.031471,0.018059,0.023556,1057.0,1057.0,1057.0,0.996230
...,...,...,...,...,...,...,...,...,...,...
504,MLSPQALFGALVLLSAPSAVV,21,0.000000,0.000000,0.000000,0.000000,419.5,838.0,419.5,0.395382
505,MRSLSSIALLSVVGAAS,17,0.000000,0.000000,0.000000,0.000000,419.5,838.0,419.5,0.395382
506,MKFFIPCIAAIFATGVLAAPTPDASLNVPLDK,32,0.000000,0.000000,0.000000,0.000000,419.5,838.0,419.5,0.395382
507,MRGLRFIPVIASLGSVFIPQSL,22,0.000000,0.000000,0.000000,0.000000,419.5,838.0,419.5,0.395382


In [39]:
# Choosing which coloums to train on 
new_input_for_ml = df_ML[['Peptide sequences','end_pos', 'MM mean abundances (Raw-Normalized)', 'MM + Nitrogen mean abundances (Raw-Normalized)']]
new_input_for_ml.columns = ['Peptide sequences','end_pos', 'MM mean abundances (Raw-Normalized)', 'MM + Nitrogen mean abundances (Raw-Normalized)']

In [40]:
df_test = h2o.H2OFrame(pd.concat([new_input_for_ml], axis='columns'))
df_test.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:1061
Cols:4

,Peptide sequences,end_pos,MM mean abundances (Raw-Normalized),MM + Nitrogen mean abundances (Raw-Normalized)
type,string,int,real,real
mins,NaN,12.0,0.0,0.0
mean,NaN,21.388312912346827,0.0011876341121270489,0.0031577579359104405
maxs,NaN,68.0,1.0,1.0
sigma,NaN,6.170686567723875,0.030737297777455288,0.03730967489556193
zeros,0,0,838,819
missing,0,0,0,0
0,MMVAWWSLFLYGLQVAAPAL,20.0,1.0,1.0
1,MVSFSSCLRALALGSSVLAVQPVL,24.0,0.0258718845992841,0.2183307968917341
2,MMVAWWSLFLYGLQVAAPAL,20.0,0.0250621360054633,0.017628084517868


In [41]:
# Select the columns we want to train on
feature_cols = ['Peptide sequences', 'end_pos']

# Initialize H2O autoML class
AutoML = H2OAutoML(
    max_runtime_secs=0,  # 1 hour =int(3600 * 1) , if unlimited time is wanted then set this to zero = 0
    max_models=None,  # None =  no limit
    nfolds=10,         # number of folds for k-fold cross-validation (nfolds=0 disables cross-validation)
    seed=1,            # Reproducibility
#    exclude_algos = ["StackedEnsemble"],
    sort_metric = "MAE",
    keep_cross_validation_predictions=True 
)

In [42]:
%%time

AutoML.train(
     x = feature_cols,
     y = 'MM mean abundances (Raw-Normalized)',
     training_frame = df_test
 ) 

AutoML progress: |
23:36:06.655: _train param, Dropping bad and constant columns: [Peptide sequences]


23:36:08.127: _train param, Dropping bad and constant columns: [Peptide sequences]


23:36:08.455: _train param, Dropping bad and constant columns: [Peptide sequences]

█
23:36:08.985: _train param, Dropping unused columns: [Peptide sequences]
23:36:09.107: _train param, Dropping bad and constant columns: [Peptide sequences]

█
23:36:10.148: _train param, Dropping bad and constant columns: [Peptide sequences]

██
23:36:11.315: _train param, Dropping bad and constant columns: [Peptide sequences]
23:36:12.29: _train param, Dropping bad and constant columns: [Peptide sequences]

███
23:36:13.40: _train param, Dropping bad and constant columns: [Peptide sequences]
23:36:13.992: _train param, Dropping unused columns: [Peptide sequences]
23:36:14.208: _train param, Dropping unused columns: [Peptide sequences]

██
23:36:14.436: _train param, Dropping bad and constant columns: [Peptide seque

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_2_20230214_233606_model_1


Status of Neuron Layers: predicting MM mean abundances (Raw-Normalized), regression, gaussian distribution, Quadratic loss, 10,401 weights/biases, 127.8 KB, 625,990 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate             rate_rms             momentum    mean_weight            weight_rms           mean_bias            bias_rms
--  -------  -------  ----------------  ---------  ----  ----  --------------------  -------------------  ----------  ---------------------  -------------------  -------------------  -----------------------
    1        1        Input             15.0
    2        100      RectifierDropout  10.0       0.0   0.0   0.9231518188118935    0.1574658751487732   0.0         -0.005691471556201577  0.1099056601524353   0.6872715216427736   0.9658355712890625
    3        100      RectifierDropout  10.0       0.0   0.0   0.33749259810640797   0.3836768865585327   0.0         -0.026951414080435825  0.10069712996482849  0.9289444135134615   0.03060510754585266
    4        1        Linear                       0.0   0.0   0.025915912502168795  0.03115076571702957  0.0         0.0015330001446636743  0.07970157265663147  -0.0413325896639707  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.0009451191202260748
RMSE: 0.03074278972744788
MAE: 0.00124611770520598
RMSLE: 0.021340994164778616
Mean Residual Deviance: 0.0009451191202260748

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.0009451008367877406
RMSE: 0.030742492364604086
MAE: 0.0016375996331657592
RMSLE: 0.021343281576380678
Mean Residual Deviance: 0.0009451008367877406

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.00162998  0.00287651  0.00971372    0.000643545   0.000700534   0.00048646    0.00115716    0.000267885   0.00144262    0.000324626   0.000181644   0.0013816
mean_residual_deviance  0.00093718  0.0029529   0.00934129    6.83073e-06   6.67242e-06   3.95354e-06   3.83102e-06   1.84474e-06   2.91373e-06   1.16505e-06   9.83129e-07   2.31856e-06
mse                     0.00093718  0.0029529   0.00934129    6.83073e-06   6.67242e-06   3.95354e-06   3.83102e-06   1.84474e-06   2.91373e-06   1.16505e-06   9.83129e-07   2.31856e-06
r2                      -0.294871   0.562388    -0.00917881   -0.00138547   -0.0546811    -0.00322681   -0.162518     -0.0400892    -1.00975      -0.00224846   -0.0333969    -1.63224
residual_deviance       0.00093718  0.0029529   0.00934129    6.83073e-06   6.67242e-06   3.95354e-06   3.83102e-06   1.84474e-06   2.91373e-06   1.16505e-06   9.83129e-07   2.31856e-06
rmse                    0.0112451   0.0300135   0.0966503     0.00261357    0.0025831     0.00198835    0.0019573     0.00135821    0.00170697    0.00107937    0.000991529   0.00152268
rmsle                   0.00826736  0.0206393   0.0669873     0.00258246    0.00255455    0.0019705     0.00194366    0.0013506     0.00170247    0.00107451    0.000987498   0.00152001

Scoring History: 
    timestamp            duration          training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-02-15 00:21:55  0.000 sec                           0         0             0          nan   

In [43]:
## Saving the leaderboard ids
leaderboard = AutoML.leaderboard
model_ids = list(leaderboard['model_id'].as_data_frame().iloc[:,0])

In [44]:
# how to save any model
out_path = os.getcwd() + '/sample_data/09-AutoML/best_models/first_round/'
mdl = h2o.get_model(model_ids[0])
h2o.save_model(model=mdl, path=out_path, force=True)


'/content/sample_data/09-AutoML/best_models/first_round/DeepLearning_grid_2_AutoML_2_20230214_233606_model_1'

In [45]:
# saving the leaderboard
out_path = os.getcwd() + '/sample_data/09-AutoML/best_models/first_round/'
h2o.export_file(leaderboard, os.path.join(out_path, 'aml_leaderboard.h2o'), force=True)

Export File progress: |██████████████████████████████████████████████████████████| (done) 100%


In [46]:
# saving the models
leaderboard = AutoML.leaderboard
lb= leaderboard
model_ids = list(leaderboard['model_id'].as_data_frame().iloc[:,0])

out_path = os.getcwd() + '/sample_data/09-AutoML/best_models/first_round/'
for m_id in model_ids:
     mdl = h2o.get_model(m_id)
     h2o.save_model(model=mdl, path=out_path, force=True)

In [47]:
len(model_ids)

284

In [48]:
# Extract the best model
df_from_h2o_object = leaderboard.as_data_frame(use_pandas=True, header=True)
df_from_h2o_object

,model_id,mae,rmse,mse,rmsle,mean_residual_deviance
0,DeepLearning_grid_2_AutoML_2_20230214_233606_m...,1.637600e-03,0.030742,0.000945,0.021343,0.000945
1,DeepLearning_grid_1_AutoML_2_20230214_233606_m...,1.657684e-03,0.030748,0.000945,0.021348,0.000945
2,DeepLearning_grid_1_AutoML_2_20230214_233606_m...,1.659860e-03,0.030736,0.000945,0.021338,0.000945
3,GBM_grid_1_AutoML_2_20230214_233606_model_61,1.774379e-03,0.030755,0.000946,0.021359,0.000946
4,XGBoost_grid_1_AutoML_2_20230214_233606_model_30,1.835588e-03,0.030776,0.000947,0.021393,0.000947
...,...,...,...,...,...,...
279,DeepLearning_grid_3_AutoML_2_20230214_233606_m...,4.397885e-03,0.030952,0.000958,0.021638,0.000958
280,DeepLearning_grid_2_AutoML_2_20230214_233606_m...,5.034195e-03,0.031036,0.000963,0.021759,0.000963
281,DeepLearning_grid_3_AutoML_2_20230214_233606_m...,1.435344e-02,0.038622,0.001492,0.031176,0.001492
282,DeepLearning_grid_3_AutoML_2_20230214_233606_m...,2.419672e-02,0.051039,0.002605,0.044489,0.002605


In [49]:
best_model = AutoML.get_best_model()
best_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_2_20230214_233606_model_1


Status of Neuron Layers: predicting MM mean abundances (Raw-Normalized), regression, gaussian distribution, Quadratic loss, 10,401 weights/biases, 127.8 KB, 625,990 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate             rate_rms             momentum    mean_weight            weight_rms           mean_bias            bias_rms
--  -------  -------  ----------------  ---------  ----  ----  --------------------  -------------------  ----------  ---------------------  -------------------  -------------------  -----------------------
    1        1        Input             15.0
    2        100      RectifierDropout  10.0       0.0   0.0   0.9231518188118935    0.1574658751487732   0.0         -0.005691471556201577  0.1099056601524353   0.6872715216427736   0.9658355712890625
    3        100      RectifierDropout  10.0       0.0   0.0   0.33749259810640797   0.3836768865585327   0.0         -0.026951414080435825  0.10069712996482849  0.9289444135134615   0.03060510754585266
    4        1        Linear                       0.0   0.0   0.025915912502168795  0.03115076571702957  0.0         0.0015330001446636743  0.07970157265663147  -0.0413325896639707  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.0009451191202260748
RMSE: 0.03074278972744788
MAE: 0.00124611770520598
RMSLE: 0.021340994164778616
Mean Residual Deviance: 0.0009451191202260748

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.0009451008367877406
RMSE: 0.030742492364604086
MAE: 0.0016375996331657592
RMSLE: 0.021343281576380678
Mean Residual Deviance: 0.0009451008367877406

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.00162998  0.00287651  0.00971372    0.000643545   0.000700534   0.00048646    0.00115716    0.000267885   0.00144262    0.000324626   0.000181644   0.0013816
mean_residual_deviance  0.00093718  0.0029529   0.00934129    6.83073e-06   6.67242e-06   3.95354e-06   3.83102e-06   1.84474e-06   2.91373e-06   1.16505e-06   9.83129e-07   2.31856e-06
mse                     0.00093718  0.0029529   0.00934129    6.83073e-06   6.67242e-06   3.95354e-06   3.83102e-06   1.84474e-06   2.91373e-06   1.16505e-06   9.83129e-07   2.31856e-06
r2                      -0.294871   0.562388    -0.00917881   -0.00138547   -0.0546811    -0.00322681   -0.162518     -0.0400892    -1.00975      -0.00224846   -0.0333969    -1.63224
residual_deviance       0.00093718  0.0029529   0.00934129    6.83073e-06   6.67242e-06   3.95354e-06   3.83102e-06   1.84474e-06   2.91373e-06   1.16505e-06   9.83129e-07   2.31856e-06
rmse                    0.0112451   0.0300135   0.0966503     0.00261357    0.0025831     0.00198835    0.0019573     0.00135821    0.00170697    0.00107937    0.000991529   0.00152268
rmsle                   0.00826736  0.0206393   0.0669873     0.00258246    0.00255455    0.0019705     0.00194366    0.0013506     0.00170247    0.00107451    0.000987498   0.00152001

Scoring History: 
    timestamp            duration          training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-02-15 00:21:55  0.000 sec                           0         0             0          nan   

In [50]:
# Get the predictions
cross_validation_holdout_predictions = best_model.cross_validation_holdout_predictions()

# make the df_test to a dataframe
as_data_frame_df_test = df_test.as_data_frame()

# Make cv_pred to a dataframe
as_data_frame_CV_predictions = cross_validation_holdout_predictions.as_data_frame()
as_data_frame_CV_predictions.columns = ['cv_holdout_predictions']
as_data_frame_CV_predictions

,cv_holdout_predictions
0,2.790091e-04
1,3.240486e-04
2,1.705240e-04
3,2.187376e-04
4,1.017495e-03
...,...
1056,1.343757e-03
1057,1.770666e-04
1058,6.419855e-07
1059,1.470384e-03


In [52]:
# merge with the test dataframe
df3 = pd.merge(as_data_frame_df_test, as_data_frame_CV_predictions, left_index=True, right_index=True)
df3 = df3.sort_values(by= 'MM mean abundances (Raw-Normalized)', ascending= False)
df3 = df3.reset_index()
df3['ranked_pct'] = df3['MM mean abundances (Raw-Normalized)'].rank(pct= True)
df3['cv_holdout_predictions_ranked_pct'] = df3['cv_holdout_predictions'].rank(pct= True)
df3

,index,Peptide sequences,end_pos,MM mean abundances (Raw-Normalized),MM + Nitrogen mean abundances (Raw-Normalized),cv_holdout_predictions,ranked_pct,cv_holdout_predictions_ranked_pct
0,0,MMVAWWSLFLYGLQVAAPAL,20,1.000000,1.000000,0.000279,1.000000,0.511310
1,1,MVSFSSCLRALALGSSVLAVQPVL,24,0.025872,0.218331,0.000324,0.999057,0.631480
2,2,MMVAWWSLFLYGLQVAAPAL,20,0.025062,0.017628,0.000171,0.998115,0.295476
3,3,MYFNYAAATLAALLPLCS,18,0.019565,0.019066,0.000219,0.997172,0.395853
4,4,MFIKYALPALAAAQAVF,17,0.018059,0.023556,0.001017,0.996230,0.756833
...,...,...,...,...,...,...,...,...
1056,504,MAQLQYLFAFILVQLQLAG,19,0.000000,0.000000,0.001019,0.395382,0.782281
1057,505,MQVKLFYTLALWAPILVS,18,0.000000,0.000000,-0.000054,0.395382,0.036758
1058,506,MKSLIWALPFIPLAY,15,0.000000,0.000000,0.001605,0.395382,0.996701
1059,507,MWPTRSLSSLFFLSLALGSPVS,22,0.000000,0.000000,0.000190,0.395382,0.351555
